# Cartes statiques 

On réalise ici des cartes sur les données statiques des bornes de recharge ainsi que la représentation de l'heuristique finale à Paris.

Importation des bibliothèques et du dataset

In [4]:
import pandas as pd 
import numpy as np
import folium

In [5]:
df_real_time=pd.read_csv("csv/ytrain_NpxebDC.csv")
df_real_time.head(20)
df_real_time.dropna()

df_static=pd.read_csv("csv/info_static.csv")
df_static

/tmp/ipykernel_36478/3739266643.py:1: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types. Specify dtype option on import or set low_memory=False.
  df_real_time=pd.read_csv("csv/ytrain_NpxebDC.csv")


,t_id,s_id,latitude,longitude,street,city,zipcode,model
0,S84-T1,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"E/F, Combo, T2, CHAdeMO, T3"
1,S84-T3,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"CHAdeMO, T2, T3, E/F, Combo"
2,S86-T1,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"CHAdeMO, Combo, T2, T3, E/F"
3,S86-T2,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,E/F
4,S86-T3,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"T3, Combo, CHAdeMO, E/F, T2"
...,...,...,...,...,...,...,...,...
268,S41-T2,S41,48.869370,2.303269,55 RUE PIERRE CHARRON,Paris,75008,E/F
269,S7-T2,S7,48.853445,2.386880,206 Boulevard Voltaire,Paris,75011,E/F
270,S57-T2,S57,48.885689,2.297463,4 AV GOURGAUD,Paris,75017,E/F
271,S47-T2,S47,48.820007,2.365661,107 BOULEVARD MASSENA,Paris,75013,E/F


In [6]:
distinct_elmts=df_real_time.apply(lambda col : col.unique())
df_real_time.replace("nan",np.nan,inplace=True)
count_nan=df_real_time.isnull().sum()

stations_list=df_static.apply(lambda col : col.unique())
stations_list["street"]

array(['104 RUE LOUIS LUMIERE', '27 RUE ORDENER',
       '150 RUE DU FAUBOURG SAINT MARTIN', '5 RUE DE REAUMUR',
       '4 Rue Changarnier', '9 PLACE DU COLONEL FABIEN', '47 RUE DIDOT',
       '55 RUE MONGE', '4 PLACE ARMAND CARREL', '5 Rue Docteur Goujon',
       '103 RUE CAULAINCOURT', '25 RUE DE VOUILLE',
       '36 AVENUE DES TERNES', '35 RUE DU BAC', '150 RUE DES POISSONIERS',
       '81 RUE DE ROME', '54 Rue Baron Leroy', '4 QUAI DU MARCHE NEUF',
       '66 RUE DE RENNES', '4 AVENUE MARCEL DORET',
       '137 BD VINCENT AURIOL', '7 BOULEVARD DES CAPUCINES',
       '3 AVENUE DU GENERAL DODDS', '10 Avenue de Flandre',
       '24 AVENUE PAUL DOUMER', '2 Rue Antonin Mercié',
       '14 rue de Crimée', '77 AVENUE DE LA GRANDE ARMEE',
       '45 AVENUE VICTOR HUGO', '9 AVENUE DE VILLARS', '22 RUE FABERT',
       '55 RUE DESNOUETTES', '21 Rue Berbier du Mets', '3 AV DUTUIT',
       '126 BOULEVARD DIDEROT', '46 AVENUE REILLE', '2 Rue Abel',
       '2 RUE PERIGNON', '44 RUE ETIENNE MARCEL

Importing Kmeans, KNN functions

In [7]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split



Indicator for average availability

In [8]:
R= 6371.0 #radius of the Earth in km
def haversine(lat1,lon1,lat2,lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.atan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c

    return distance




In [9]:
radius=1 #check for availability in a 1 km radius
lat_stations=df_static["latitude"].unique()
lon_stations=df_static["longitude"].unique()
coord_stations=np.column_stack((lat_stations,lon_stations))

coord_stations

array([[48.863766 ,  2.41177  ],
       [48.89061  ,  2.35423  ],
       [48.875465 ,  2.360103 ],
       [48.864719 ,  2.359043 ],
       [48.8452653,  2.4112383],
       [48.878799 ,  2.368851 ],
       [48.831448 ,  2.320052 ],
       [48.84477  ,  2.35242  ],
       [48.882794 ,  2.382705 ],
       [48.839947 ,  2.397822 ],
       [48.889731 ,  2.33891  ],
       [48.834705 ,  2.305529 ],
       [48.878965 ,  2.293606 ],
       [48.857124 ,  2.327753 ],
       [48.897671 ,  2.352758 ],
       [48.882718 ,  2.319768 ],
       [48.833603 ,  2.389173 ],
       [48.854057 ,  2.34556  ],
       [48.8521255,  2.3313459],
       [48.837055 ,  2.259932 ],
       [48.832893 ,  2.363025 ],
       [48.870628 ,  2.33321  ],
       [48.834084 ,  2.405248 ],
       [48.8847205,  2.3699612],
       [48.861016 ,  2.28308  ],
       [48.828454 ,  2.3006428],
       [48.8778   ,  2.39182  ],
       [48.876162 ,  2.286403 ],
       [48.871465 ,  2.289894 ],
       [48.852665 ,  2.31368  ],
       [48

In [10]:
import folium
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
from math import radians, sin, cos, sqrt, atan2
import matplotlib.pyplot as plt
import geopandas as gpd

In [11]:


# Charger le fichier GeoJSON des arrondissements de Paris
arrondissements_geojson = "arrondissements.geojson"  
arrondissements_gdf = gpd.read_file(arrondissements_geojson)  # Charger les arrondissements


In [12]:
# Fonction haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [13]:

# Fonction `stations_distance`
def stations_distance(lat, lon):
    df_static['distance'] = df_static.apply(lambda row: haversine(lat, lon, row['latitude'], row['longitude']), axis=1)
    df_sorted = df_static.sort_values(by='distance', ascending=True)
    return df_sorted['distance'].iloc[0]  # Distance minimale

In [14]:
# Carte Folium
m2 = folium.Map(location=(48.863766, 2.41177))
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
latitudes = np.arange(lat_min, lat_max, 0.005)
longitudes = np.arange(lon_min, lon_max, 0.005)

In [15]:

# Fonction pour convertir une distance en une couleur
def distance_to_color(distance, max_distance):
    normalized = min(distance / max_distance, 1)
    rgba_color = plt.cm.Reds(normalized)
    return f"rgba({int(rgba_color[0] * 255)}, {int(rgba_color[1] * 255)}, {int(rgba_color[2] * 255)}, {rgba_color[3]})"

In [16]:
# Coordonnées de Charenton-le-Pont et du Jardin d'Acclimatation
charenton_lat, charenton_lon = 48.8337, 2.4149
jardin_lat, jardin_lon = 48.8716, 2.2611



# Distance d'exclusion (en kilomètres)
exclusion_radius = 1  # 2 km autour de ces points

# Fonction pour vérifier si un carré est proche d'un des lieux d'exclusion
def is_near_exclusion(lat, lon):
    distance_to_charenton = haversine(lat, lon, charenton_lat, charenton_lon)
    distance_to_jardin = haversine(lat, lon, jardin_lat, jardin_lon)
    return distance_to_charenton < exclusion_radius or distance_to_jardin < exclusion_radius


In [17]:


# Calcul des distances pour chaque carré et ajout à la carte
max_distance = 0
square_distances = []

for i in range(len(latitudes) - 1):
    for j in range(len(longitudes) - 1):
        # Centre du carré
        center_lat = (latitudes[i] + latitudes[i + 1]) / 2
        center_lon = (longitudes[j] + longitudes[j + 1]) / 2
        center_point = Point(center_lon, center_lat)
        
        # Vérifier si le centre est dans Paris (tous arrondissements combinés)
        if not arrondissements_gdf.geometry.unary_union.contains(center_point):
            continue  # Ignorer les carrés hors des arrondissements de Paris
        if is_near_exclusion(center_lat, center_lon):
            continue  # Ignorer les carrés proches des zones exclues
        
        # Calcul de la distance à la station la plus proche
        distance_to_station = stations_distance(center_lat, center_lon)
        square_distances.append(distance_to_station)
        max_distance = max(max_distance, distance_to_station)
        
        # Définir les limites du carré
        bounds = [
            (latitudes[i], longitudes[j]),
            (latitudes[i], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j]),
        ]
        
        # Ajouter le carré à la carte avec une couleur
        color = distance_to_color(distance_to_station, max_distance)
        folium.Polygon(
            locations=bounds,
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.5
        ).add_to(m2)


# Afficher la carte


/tmp/ipykernel_36478/1222012352.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if not arrondissements_gdf.geometry.unary_union.contains(center_point):
/tmp/ipykernel_36478/1222012352.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if not arrondissements_gdf.geometry.unary_union.contains(center_point):


In [18]:
m2


In [36]:
df_scores=pd.read_csv('csv/scores_et_coords.csv')

#Carte des différents scores

def score_to_color(indicator, max_value=10):
    normalized = min(indicator / max_value, 1)
    rgba_color = plt.cm.Reds(normalized)
    return f"rgba({int(rgba_color[0] * 255)}, {int(rgba_color[1] * 255)}, {int(rgba_color[2] * 255)}, {rgba_color[3]:.2f})"

df_scores.head(20)

,Unnamed: 0,carre_id,mean_score,lati,long
0,0,10_1,8.914754,48.8625,2.2625
1,1,10_10,4.928173,48.8625,2.3075
2,2,10_11,7.067954,48.8625,2.3125
3,3,10_12,6.637603,48.8625,2.3175
4,4,10_13,8.438791,48.8625,2.3225
5,5,10_14,5.550159,48.8625,2.3275
6,6,10_15,6.649251,48.8625,2.3325
7,7,10_16,1.012305,48.8625,2.3375
8,8,10_18,2.652704,48.8625,2.3475
9,9,10_19,5.398497,48.8625,2.3525


In [44]:

# Carte Folium
m3 = folium.Map(location=(48.863766, 2.41177))




for i in range(len(latitudes) - 1):
    for j in range(len(longitudes) - 1):
        # Centre du carré
        center_lat = (latitudes[i] + latitudes[i + 1]) / 2
        center_lon = (longitudes[j] + longitudes[j + 1]) / 2
        center_point = Point(center_lon, center_lat)
        
        # Vérifier si le centre est dans Paris (tous arrondissements combinés)
        if not arrondissements_gdf.geometry.unary_union.contains(center_point):
            continue  
        if is_near_exclusion(center_lat, center_lon):
            continue  
        
        # Calcul du score
        score_value = df_scores[(df_scores['lati'] == center_lat) & (df_scores['long'] == center_lon)]['mean_score'].values

        
        if len(score_value) > 0:
            score_value = score_value[0]  
        else:
            score_value = 0  
        
        
        # Limites du carré
        bounds = [
            (latitudes[i], longitudes[j]),
            (latitudes[i], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j]),
        ]
        
        
        color = score_to_color(score_value, max_score)
        folium.Polygon(
            locations=bounds,
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(m3)





/tmp/ipykernel_36478/1555832817.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if not arrondissements_gdf.geometry.unary_union.contains(center_point):


In [45]:
m3